In [1]:
import pandas as pd
import csv,os
import time
from datetime import datetime, date, time, timedelta


##### d1=store master
##### d2=Item Master
##### d3=Closing Stock as on_dd.mm.yy ---> Closing Stock as on_01.12.19 

In [2]:
#Define you directory
data_dir = '/Users/nirajkulkarni/Desktop/Niraj/ISB-CBA/Capstone/data/'

d1 = pd.read_excel(data_dir + 'Encoded/store master.xlsx')
d2 = pd.read_excel(data_dir + 'item-data/Item Master.xlsx')
d3 = pd.read_excel(data_dir + 'Encoded/Closing Stock as on_01.12.19.xlsx') #Closing Stock as on_01.12.19.xlsx') #')


In [3]:
#create a final datafram table
final_summ_file = pd.DataFrame()

In [4]:
#New Col- Day_in_Stock
file_week = os.path.basename(data_dir + 'Encoded/Closing Stock as on_01.12.19.xlsx' )
file_week = file_week[20:26]+'20'+file_week[26:28]
file_week = datetime.strptime(file_week, '%d.%m.%Y')
#d3_new = d3
d3['days_in_stock_init'] =  file_week - d3['Purchase Date']

In [5]:
d3.head(2)
d3[d3['Item No_'] == 5175335]

,Location Code,Item No_,Brand,Department,Quantity,Cost Amount,Purchase MRP,Prevailing MRP as on Stock Date,Purchase Date,State,Region,days_in_stock_init
207,S02,5175335,B096,WAT,-1,0.00,153300.0,176000.0,2017-04-06,ST05,West,969 days
208,S02,5175335,B096,WAT,1,0.00,153300.0,176000.0,2017-07-31,ST05,West,853 days
14631,S121,5175335,B096,WAT,1,76616.69,153300.0,176000.0,2018-03-21,ST28,East,620 days
18874,S131,5175335,B096,WAT,1,225008.71,153300.0,176000.0,2018-03-22,ST12,West,619 days
18875,S131,5175335,B096,WAT,1,225008.71,160600.0,176000.0,2018-05-11,ST12,West,569 days
18876,S131,5175335,B096,WAT,1,225008.71,153300.0,176000.0,2017-07-31,ST12,West,853 days
26627,S24,5175335,B096,WAT,1,82370.21,176000.0,176000.0,2019-03-08,ST16,South,268 days
30166,S31,5175335,B096,WAT,1,77412.62,160600.0,176000.0,2018-05-11,ST15,North,569 days
38855,S62,5175335,B096,WAT,1,79812.33,176000.0,176000.0,2019-01-16,ST02,North,319 days
40309,S66,5175335,B096,WAT,1,82212.05,176000.0,176000.0,2019-01-16,ST12,West,319 days


In [6]:
#instead of agg, have to to use this logic to avoid error of aggreagte timedelta type
days_in_stock_aggdf = d3.groupby(["Item No_","Location Code"])['days_in_stock_init'].agg(['sum', 'count']) 
days_in_stock_aggdf['days_in_stock_init'] = days_in_stock_aggdf['sum']/days_in_stock_aggdf['count']
days_in_stock_aggdf.head()

,,sum,count,days_in_stock_init
Item No_,Location Code,,,
5100311,S25,1065 days,1,1065 days
5100392,S109,1164 days,1,1164 days
5101288,S122,4262 days,1,4262 days
5101296,S122,2128 days,1,2128 days
5103985,S122,4061 days,1,4061 days


In [7]:
merged_days_in_stock_df = d3.merge(days_in_stock_aggdf, right_index=True, how = 'inner',
                                                 left_on=["Item No_","Location Code"], right_on=["Item No_","Location Code"],
                                        suffixes= ('_x', '_y')) #, indicator= True)
merged_days_in_stock_df[(merged_days_in_stock_df['Item No_'] == 5175335) & (merged_days_in_stock_df['Location Code'] == 'S02')]


,Location Code,Item No_,Brand,Department,Quantity,Cost Amount,Purchase MRP,Prevailing MRP as on Stock Date,Purchase Date,State,Region,days_in_stock_init_x,sum,count,days_in_stock_init_y
207,S02,5175335,B096,WAT,-1,0.0,153300.0,176000.0,2017-04-06,ST05,West,969 days,1822 days,2,911 days
208,S02,5175335,B096,WAT,1,0.0,153300.0,176000.0,2017-07-31,ST05,West,853 days,1822 days,2,911 days


In [8]:
#rename column
merged_days_in_stock_df.rename(columns = {'days_in_stock_init_y':'days_in_stock_agg'}, inplace = True)
d3 = merged_days_in_stock_df


In [9]:
import numpy as np

#distinct store code/location code and item no and do quantity of stock group by, or keep without duplicated?
d3_unique = d3.drop_duplicates(["Location Code","Item No_"])
d3_unique[d3_unique['Item No_'] == 5184261].count()


Location Code                      21
Item No_                           21
Brand                              21
Department                         21
Quantity                           21
Cost Amount                        21
Purchase MRP                       21
Prevailing MRP as on Stock Date    21
Purchase Date                      21
State                              21
Region                             21
days_in_stock_init_x               21
sum                                21
count                              21
days_in_stock_agg                  21
dtype: int64

In [10]:
d3_unique[d3_unique['Item No_'] == 5175335] 

,Location Code,Item No_,Brand,Department,Quantity,Cost Amount,Purchase MRP,Prevailing MRP as on Stock Date,Purchase Date,State,Region,days_in_stock_init_x,sum,count,days_in_stock_agg
207,S02,5175335,B096,WAT,-1,0.00,153300.0,176000.0,2017-04-06,ST05,West,969 days,1822 days,2,911 days 00:00:00
14631,S121,5175335,B096,WAT,1,76616.69,153300.0,176000.0,2018-03-21,ST28,East,620 days,620 days,1,620 days 00:00:00
18874,S131,5175335,B096,WAT,1,225008.71,153300.0,176000.0,2018-03-22,ST12,West,619 days,2041 days,3,680 days 08:00:00
26627,S24,5175335,B096,WAT,1,82370.21,176000.0,176000.0,2019-03-08,ST16,South,268 days,268 days,1,268 days 00:00:00
30166,S31,5175335,B096,WAT,1,77412.62,160600.0,176000.0,2018-05-11,ST15,North,569 days,569 days,1,569 days 00:00:00
38855,S62,5175335,B096,WAT,1,79812.33,176000.0,176000.0,2019-01-16,ST02,North,319 days,319 days,1,319 days 00:00:00
40309,S66,5175335,B096,WAT,1,82212.05,176000.0,176000.0,2019-01-16,ST12,West,319 days,319 days,1,319 days 00:00:00
41795,S74,5175335,B096,WAT,1,61779.03,130400.0,151400.0,2017-05-18,ST09,South,927 days,927 days,1,927 days 00:00:00
44087,S89,5175335,B096,WAT,1,165193.10,176000.0,176000.0,2019-10-15,ST19,North,47 days,366 days,2,183 days 00:00:00


In [11]:
#d3.groupby(["Location Code","Item No_"]).agg(['count'])
#stock_quantity = d3[d3['Quantity'] > 0].groupby(["Location Code","Item No_"])['Quantity'].count()
stock_quantitydf = d3.groupby(["Location Code","Item No_"])['Quantity'].agg(['sum']) #count()
#stock_quantitydf = stock_quantitydf.to_frame()
#print(type(stock_quantity))
print(type(stock_quantitydf))

<class 'pandas.core.frame.DataFrame'>


In [12]:
merge_quantity = d3_unique.merge(stock_quantitydf, right_index=True, left_on=['Location Code','Item No_'],
                                 right_on=['Location Code','Item No_'],
                                suffixes= ('_m', '_n')) 
#merge_quantity.head(4)
merge_quantity.rename(columns={'sum_n': 'stock_quantity'}, inplace=True)
merge_quantity[merge_quantity['Item No_'] == 5175335] 


,Location Code,Item No_,Brand,Department,Quantity,Cost Amount,Purchase MRP,Prevailing MRP as on Stock Date,Purchase Date,State,Region,days_in_stock_init_x,sum_m,count,days_in_stock_agg,stock_quantity
207,S02,5175335,B096,WAT,-1,0.00,153300.0,176000.0,2017-04-06,ST05,West,969 days,1822 days,2,911 days 00:00:00,0
14631,S121,5175335,B096,WAT,1,76616.69,153300.0,176000.0,2018-03-21,ST28,East,620 days,620 days,1,620 days 00:00:00,1
18874,S131,5175335,B096,WAT,1,225008.71,153300.0,176000.0,2018-03-22,ST12,West,619 days,2041 days,3,680 days 08:00:00,3
26627,S24,5175335,B096,WAT,1,82370.21,176000.0,176000.0,2019-03-08,ST16,South,268 days,268 days,1,268 days 00:00:00,1
30166,S31,5175335,B096,WAT,1,77412.62,160600.0,176000.0,2018-05-11,ST15,North,569 days,569 days,1,569 days 00:00:00,1
38855,S62,5175335,B096,WAT,1,79812.33,176000.0,176000.0,2019-01-16,ST02,North,319 days,319 days,1,319 days 00:00:00,1
40309,S66,5175335,B096,WAT,1,82212.05,176000.0,176000.0,2019-01-16,ST12,West,319 days,319 days,1,319 days 00:00:00,1
41795,S74,5175335,B096,WAT,1,61779.03,130400.0,151400.0,2017-05-18,ST09,South,927 days,927 days,1,927 days 00:00:00,1
44087,S89,5175335,B096,WAT,1,165193.10,176000.0,176000.0,2019-10-15,ST19,North,47 days,366 days,2,183 days 00:00:00,2


In [13]:
#Col 19 - Stock quantity needs to be grouped and counted
#TBD
#Col - 1
final_summ_file['Col_PK'] = merge_quantity['Item No_'].map(str) +'_'+ merge_quantity['Location Code']
final_summ_file[['Item No_', 'Location Code', "Purchase MRP", "Prevailing MRP as on Stock Date",'State','Region',
                 'stock_quantity','days_in_stock_agg']] = merge_quantity.loc[:,['Item No_', 'Location Code', 
                                                                                "Purchase MRP", "Prevailing MRP as on Stock Date",
                                                                                'State','Region','stock_quantity','days_in_stock_agg']]

final_summ_file[final_summ_file['Item No_'] == 5175335] 

,Col_PK,Item No_,Location Code,Purchase MRP,Prevailing MRP as on Stock Date,State,Region,stock_quantity,days_in_stock_agg
207,5175335_S02,5175335,S02,153300.0,176000.0,ST05,West,0,911 days 00:00:00
14631,5175335_S121,5175335,S121,153300.0,176000.0,ST28,East,1,620 days 00:00:00
18874,5175335_S131,5175335,S131,153300.0,176000.0,ST12,West,3,680 days 08:00:00
26627,5175335_S24,5175335,S24,176000.0,176000.0,ST16,South,1,268 days 00:00:00
30166,5175335_S31,5175335,S31,160600.0,176000.0,ST15,North,1,569 days 00:00:00
38855,5175335_S62,5175335,S62,176000.0,176000.0,ST02,North,1,319 days 00:00:00
40309,5175335_S66,5175335,S66,176000.0,176000.0,ST12,West,1,319 days 00:00:00
41795,5175335_S74,5175335,S74,130400.0,151400.0,ST09,South,1,927 days 00:00:00
44087,5175335_S89,5175335,S89,176000.0,176000.0,ST19,North,2,183 days 00:00:00


In [14]:
#Col-2 Week
final_summ_file['Week_Start'] = file_week

#Col -3 Month

final_summ_file['Month'] =  file_week.month

#Col- 4 Year
final_summ_file['Year'] =  file_week.year

In [15]:
final_summ_file.dtypes

Col_PK                                      object
Item No_                                     int64
Location Code                               object
Purchase MRP                               float64
Prevailing MRP as on Stock Date            float64
State                                       object
Region                                      object
stock_quantity                               int64
days_in_stock_agg                  timedelta64[ns]
Week_Start                          datetime64[ns]
Month                                        int64
Year                                         int64
dtype: object

In [16]:
final_summ_file[final_summ_file['Item No_'] == 5184261]

,Col_PK,Item No_,Location Code,Purchase MRP,Prevailing MRP as on Stock Date,State,Region,stock_quantity,days_in_stock_agg,Week_Start,Month,Year
428,5184261_S02,5184261,S02,23000.0,27000.0,ST05,West,3,364 days 08:00:00,2019-12-01,12,2019
1984,5184261_S06,5184261,S06,23000.0,27000.0,ST09,South,1,430 days 00:00:00,2019-12-01,12,2019
3712,5184261_S07,5184261,S07,23000.0,27000.0,ST09,South,2,335 days 12:00:00,2019-12-01,12,2019
4288,5184261_S10,5184261,S10,23000.0,27000.0,ST12,West,1,353 days 00:00:00,2019-12-01,12,2019
4771,5184261_S102,5184261,S102,23000.0,27000.0,ST24,North,1,391 days 00:00:00,2019-12-01,12,2019
5487,5184261_S106,5184261,S106,23000.0,27000.0,ST12,West,1,552 days 00:00:00,2019-12-01,12,2019
6323,5184261_S109,5184261,S109,25000.0,27000.0,ST12,West,1,62 days 00:00:00,2019-12-01,12,2019
11501,5184261_S111,5184261,S111,23000.0,27000.0,ST13,North,1,268 days 00:00:00,2019-12-01,12,2019
12418,5184261_S114,5184261,S114,21900.0,27000.0,ST17,South,2,447 days 00:00:00,2019-12-01,12,2019
18181,5184261_S127,5184261,S127,25000.0,27000.0,ST21,East,1,151 days 00:00:00,2019-12-01,12,2019


In [17]:
d1.head()

,Store Code,Store type,Store location,City type,State,Region
0,S01,Ethos,Mall,Tier2,NaN,NaN
1,S02,Ethos,Mall,Tier2,NaN,NaN
2,S03,Ethos,Mall,Tier1,NaN,NaN
3,S04,Ethos,DFS,Tier1,NaN,NaN
4,S05,Ethos,DFS,Tier1,NaN,NaN


In [18]:
merged_store_df = final_summ_file.merge(d1[['Store Code','Store type','Store location','City type','State','Region']],
                                        how = 'inner', left_on=["Location Code"], right_on=["Store Code"],
                                        suffixes= ('_m', '_n')) #, indicator= True)



merged_store_df.head()

,Col_PK,Item No_,Location Code,Purchase MRP,Prevailing MRP as on Stock Date,State_m,Region_m,stock_quantity,days_in_stock_agg,Week_Start,Month,Year,Store Code,Store type,Store location,City type,State_n,Region_n
0,5106817_S02,5106817,S02,63140.0,66300.0,ST05,West,1,2166 days,2019-12-01,12,2019,S02,Ethos,Mall,Tier2,NaN,NaN
1,5107206_S02,5107206,S02,142500.0,154100.0,ST05,West,1,261 days,2019-12-01,12,2019,S02,Ethos,Mall,Tier2,NaN,NaN
2,5107230_S02,5107230,S02,215000.0,232600.0,ST05,West,1,261 days,2019-12-01,12,2019,S02,Ethos,Mall,Tier2,NaN,NaN
3,5107277_S02,5107277,S02,78000.0,87600.0,ST05,West,1,1314 days,2019-12-01,12,2019,S02,Ethos,Mall,Tier2,NaN,NaN
4,5107397_S02,5107397,S02,64500.0,69400.0,ST05,West,1,229 days,2019-12-01,12,2019,S02,Ethos,Mall,Tier2,NaN,NaN


In [19]:
final_summ_file  = merged_store_df
final_summ_file[final_summ_file['Item No_'] == 5184261].count()

Col_PK                             21
Item No_                           21
Location Code                      21
Purchase MRP                       21
Prevailing MRP as on Stock Date    21
State_m                            21
Region_m                           21
stock_quantity                     21
days_in_stock_agg                  21
Week_Start                         21
Month                              21
Year                               21
Store Code                         21
Store type                         21
Store location                     21
City type                          21
State_n                             0
Region_n                            0
dtype: int64

In [20]:
final_summ_file[final_summ_file['Item No_'] == 5184261]

,Col_PK,Item No_,Location Code,Purchase MRP,Prevailing MRP as on Stock Date,State_m,Region_m,stock_quantity,days_in_stock_agg,Week_Start,Month,Year,Store Code,Store type,Store location,City type,State_n,Region_n
389,5184261_S02,5184261,S02,23000.0,27000.0,ST05,West,3,364 days 08:00:00,2019-12-01,12,2019,S02,Ethos,Mall,Tier2,NaN,NaN
1734,5184261_S06,5184261,S06,23000.0,27000.0,ST09,South,1,430 days 00:00:00,2019-12-01,12,2019,S06,Ethos,DFS,Tier1,NaN,NaN
3170,5184261_S07,5184261,S07,23000.0,27000.0,ST09,South,2,335 days 12:00:00,2019-12-01,12,2019,S07,Ethos,DFS,Tier1,NaN,NaN
3721,5184261_S10,5184261,S10,23000.0,27000.0,ST12,West,1,353 days 00:00:00,2019-12-01,12,2019,S10,Ethos,Mall,Tier1,NaN,NaN
4185,5184261_S102,5184261,S102,23000.0,27000.0,ST24,North,1,391 days 00:00:00,2019-12-01,12,2019,S102,Ethos,Mall,Tier2,NaN,NaN
4839,5184261_S106,5184261,S106,23000.0,27000.0,ST12,West,1,552 days 00:00:00,2019-12-01,12,2019,S106,Ethos,Mall,Tier2,NaN,NaN
5573,5184261_S109,5184261,S109,25000.0,27000.0,ST12,West,1,62 days 00:00:00,2019-12-01,12,2019,S109,Ethos,Mall,Tier2,NaN,NaN
9357,5184261_S111,5184261,S111,23000.0,27000.0,ST13,North,1,268 days 00:00:00,2019-12-01,12,2019,S111,Ethos,Mall,Tier2,NaN,NaN
10178,5184261_S114,5184261,S114,21900.0,27000.0,ST17,South,2,447 days 00:00:00,2019-12-01,12,2019,S114,Ethos,Mall,Tier2,NaN,NaN
15414,5184261_S127,5184261,S127,25000.0,27000.0,ST21,East,1,151 days 00:00:00,2019-12-01,12,2019,S127,Large,High Street,Tier2,NaN,NaN


In [21]:
d2.head(3)

,No_,Brand,Department,Movement,Case Material,Watchband _ Strap Type,Dial Color,Case Shape,Case Size
0,5100010,B088,WAT,QUARTZ,STEEL,STEEL,OTHERS,NaN,37 MM
1,5100011,B081,WAT,OTHERS,OTHERS,OTHERS,OTHERS,NaN,NaN
2,5100012,B081,WAT,OTHERS,OTHERS,OTHERS,OTHERS,NaN,NaN


In [22]:
merged_item_df = final_summ_file.merge(d2[['No_','Brand','Movement','Case Material','Watchband _ Strap Type','Dial Color','Case Shape','Case Size']],
                                        how = 'inner', left_on=["Item No_"], right_on=["No_"],
                                        suffixes= ('_m', '_n')) #, indicator= True)



merged_item_df.head()



,Col_PK,Item No_,Location Code,Purchase MRP,Prevailing MRP as on Stock Date,State_m,Region_m,stock_quantity,days_in_stock_agg,Week_Start,...,State_n,Region_n,No_,Brand,Movement,Case Material,Watchband _ Strap Type,Dial Color,Case Shape,Case Size
0,5106817_S02,5106817,S02,63140.0,66300.0,ST05,West,1,2166 days,2019-12-01,...,NaN,NaN,5106817,B063,QUARTZ,STEEL,STEEL,WHITE,RECTANGULAR,32 mm
1,5106817_S23,5106817,S23,62500.0,66300.0,ST12,West,1,2358 days,2019-12-01,...,NaN,NaN,5106817,B063,QUARTZ,STEEL,STEEL,WHITE,RECTANGULAR,32 mm
2,5106817_S24,5106817,S24,48100.0,66300.0,ST16,South,1,2128 days,2019-12-01,...,NaN,NaN,5106817,B063,QUARTZ,STEEL,STEEL,WHITE,RECTANGULAR,32 mm
3,5106817_S62,5106817,S62,56500.0,66300.0,ST02,North,1,2305 days,2019-12-01,...,NaN,NaN,5106817,B063,QUARTZ,STEEL,STEEL,WHITE,RECTANGULAR,32 mm
4,5107206_S02,5107206,S02,142500.0,154100.0,ST05,West,1,261 days,2019-12-01,...,NaN,NaN,5107206,B063,AUTOMATIC,STEEL,LEATHER,SILVER,ROUND,40 mm


In [23]:
final_summ_file  = merged_item_df
merged_item_df[merged_item_df['Item No_'] == 5106817]

,Col_PK,Item No_,Location Code,Purchase MRP,Prevailing MRP as on Stock Date,State_m,Region_m,stock_quantity,days_in_stock_agg,Week_Start,...,State_n,Region_n,No_,Brand,Movement,Case Material,Watchband _ Strap Type,Dial Color,Case Shape,Case Size
0,5106817_S02,5106817,S02,63140.0,66300.0,ST05,West,1,2166 days,2019-12-01,...,NaN,NaN,5106817,B063,QUARTZ,STEEL,STEEL,WHITE,RECTANGULAR,32 mm
1,5106817_S23,5106817,S23,62500.0,66300.0,ST12,West,1,2358 days,2019-12-01,...,NaN,NaN,5106817,B063,QUARTZ,STEEL,STEEL,WHITE,RECTANGULAR,32 mm
2,5106817_S24,5106817,S24,48100.0,66300.0,ST16,South,1,2128 days,2019-12-01,...,NaN,NaN,5106817,B063,QUARTZ,STEEL,STEEL,WHITE,RECTANGULAR,32 mm
3,5106817_S62,5106817,S62,56500.0,66300.0,ST02,North,1,2305 days,2019-12-01,...,NaN,NaN,5106817,B063,QUARTZ,STEEL,STEEL,WHITE,RECTANGULAR,32 mm


In [24]:
final_summ_file[final_summ_file['Item No_'] == 5184261].count()

Col_PK                             0
Item No_                           0
Location Code                      0
Purchase MRP                       0
Prevailing MRP as on Stock Date    0
State_m                            0
Region_m                           0
stock_quantity                     0
days_in_stock_agg                  0
Week_Start                         0
Month                              0
Year                               0
Store Code                         0
Store type                         0
Store location                     0
City type                          0
State_n                            0
Region_n                           0
No_                                0
Brand                              0
Movement                           0
Case Material                      0
Watchband _ Strap Type             0
Dial Color                         0
Case Shape                         0
Case Size                          0
dtype: int64

In [25]:
final_summ_file[final_summ_file['Item No_'] == 5106817]

,Col_PK,Item No_,Location Code,Purchase MRP,Prevailing MRP as on Stock Date,State_m,Region_m,stock_quantity,days_in_stock_agg,Week_Start,...,State_n,Region_n,No_,Brand,Movement,Case Material,Watchband _ Strap Type,Dial Color,Case Shape,Case Size
0,5106817_S02,5106817,S02,63140.0,66300.0,ST05,West,1,2166 days,2019-12-01,...,NaN,NaN,5106817,B063,QUARTZ,STEEL,STEEL,WHITE,RECTANGULAR,32 mm
1,5106817_S23,5106817,S23,62500.0,66300.0,ST12,West,1,2358 days,2019-12-01,...,NaN,NaN,5106817,B063,QUARTZ,STEEL,STEEL,WHITE,RECTANGULAR,32 mm
2,5106817_S24,5106817,S24,48100.0,66300.0,ST16,South,1,2128 days,2019-12-01,...,NaN,NaN,5106817,B063,QUARTZ,STEEL,STEEL,WHITE,RECTANGULAR,32 mm
3,5106817_S62,5106817,S62,56500.0,66300.0,ST02,North,1,2305 days,2019-12-01,...,NaN,NaN,5106817,B063,QUARTZ,STEEL,STEEL,WHITE,RECTANGULAR,32 mm


In [26]:
final_summ_file.columns

Index(['Col_PK', 'Item No_', 'Location Code', 'Purchase MRP',
       'Prevailing MRP as on Stock Date', 'State_m', 'Region_m',
       'stock_quantity', 'days_in_stock_agg', 'Week_Start', 'Month', 'Year',
       'Store Code', 'Store type', 'Store location', 'City type', 'State_n',
       'Region_n', 'No_', 'Brand', 'Movement', 'Case Material',
       'Watchband _ Strap Type', 'Dial Color', 'Case Shape', 'Case Size'],
      dtype='object')

In [27]:
final_summ_file.dtypes

Col_PK                                      object
Item No_                                     int64
Location Code                               object
Purchase MRP                               float64
Prevailing MRP as on Stock Date            float64
State_m                                     object
Region_m                                    object
stock_quantity                               int64
days_in_stock_agg                  timedelta64[ns]
Week_Start                          datetime64[ns]
Month                                        int64
Year                                         int64
Store Code                                  object
Store type                                  object
Store location                              object
City type                                   object
State_n                                    float64
Region_n                                   float64
No_                                          int64
Brand                          

In [28]:
#import datetime

#final_summ_file['week_time'] =  pd.to_datetime(final_summ_file['Week'], format='%d.%m.%Y')

In [29]:
final_summ_file.head()

,Col_PK,Item No_,Location Code,Purchase MRP,Prevailing MRP as on Stock Date,State_m,Region_m,stock_quantity,days_in_stock_agg,Week_Start,...,State_n,Region_n,No_,Brand,Movement,Case Material,Watchband _ Strap Type,Dial Color,Case Shape,Case Size
0,5106817_S02,5106817,S02,63140.0,66300.0,ST05,West,1,2166 days,2019-12-01,...,NaN,NaN,5106817,B063,QUARTZ,STEEL,STEEL,WHITE,RECTANGULAR,32 mm
1,5106817_S23,5106817,S23,62500.0,66300.0,ST12,West,1,2358 days,2019-12-01,...,NaN,NaN,5106817,B063,QUARTZ,STEEL,STEEL,WHITE,RECTANGULAR,32 mm
2,5106817_S24,5106817,S24,48100.0,66300.0,ST16,South,1,2128 days,2019-12-01,...,NaN,NaN,5106817,B063,QUARTZ,STEEL,STEEL,WHITE,RECTANGULAR,32 mm
3,5106817_S62,5106817,S62,56500.0,66300.0,ST02,North,1,2305 days,2019-12-01,...,NaN,NaN,5106817,B063,QUARTZ,STEEL,STEEL,WHITE,RECTANGULAR,32 mm
4,5107206_S02,5107206,S02,142500.0,154100.0,ST05,West,1,261 days,2019-12-01,...,NaN,NaN,5107206,B063,AUTOMATIC,STEEL,LEATHER,SILVER,ROUND,40 mm


In [30]:
final_summ_file.dtypes

Col_PK                                      object
Item No_                                     int64
Location Code                               object
Purchase MRP                               float64
Prevailing MRP as on Stock Date            float64
State_m                                     object
Region_m                                    object
stock_quantity                               int64
days_in_stock_agg                  timedelta64[ns]
Week_Start                          datetime64[ns]
Month                                        int64
Year                                         int64
Store Code                                  object
Store type                                  object
Store location                              object
City type                                   object
State_n                                    float64
Region_n                                   float64
No_                                          int64
Brand                          

In [31]:
#read all sales file
sale_17_18 = pd.read_excel(data_dir + 'item-data/2017-18/Sale Data 01.04.17 to 31.03.18.xlsx')
sale_17_18.columns = ['Store No','Date','Item No', 'Brand','Department','Customer No', 'Quantity','Price',
                      'Total Price','Line Discount Amount', 'CRM Line Disc_ Amount', 'Discount Amount',
                      'Tax Amount' ,'Cost Amount', 'Billing', 'Contribution', 'Receipt Date','Trade Incentive' ,
                      'Trade Incentives Value', 'Total Contribution', 'State','Region'] 

In [32]:
sale_18_19 = pd.read_excel(data_dir + 'item-data/2018-19/Sale Data 01.04.18 to 31.03.19.xlsx')
sale_18_19.columns = ['Store No','Date','Item No', 'Brand','Department','Customer No', 'Quantity','Price',
                      'Total Price','Line Discount Amount', 'CRM Line Disc_ Amount', 'Discount Amount',
                      'Tax Amount' ,'Cost Amount', 'Billing', 'Contribution', 'Receipt Date','Trade Incentive' ,
                      'Trade Incentives Value', 'Total Contribution', 'State','Region'] 

In [33]:
sale_19_20 = pd.read_excel(data_dir + 'item-data/2019-20/Sale Data 01.04.19 to 06.02.2020.xlsx') 
sale_19_20.columns = ['Store No','Date','Item No', 'Brand','Department','Customer No', 'Quantity','Price',
                      'Total Price','Line Discount Amount', 'CRM Line Disc_ Amount', 'Discount Amount',
                      'Tax Amount' ,'Cost Amount', 'Billing', 'Contribution', 'Receipt Date','Trade Incentive' ,
                      'Trade Incentives Value', 'Total Contribution', 'State','Region'] 


In [34]:
print(sale_17_18.shape)
print(sale_18_19.shape)
print(sale_19_20.shape)

(75181, 22)
(76956, 22)
(64530, 22)


In [35]:
#concat all sales file
all_sales_list = [sale_17_18,sale_18_19,sale_19_20]
all_sales = pd.concat(all_sales_list)

In [36]:
all_sales.shape

(216667, 22)

In [37]:
all_sales.head()

,Store No,Date,Item No,Brand,Department,Customer No,Quantity,Price,Total Price,Line Discount Amount,...,Tax Amount,Cost Amount,Billing,Contribution,Receipt Date,Trade Incentive,Trade Incentives Value,Total Contribution,State,Region
0,S02,2017-09-30,5161847,B074,WAT,CU00247184,1,312600.0,312600.0,93800.0,...,47862.50,163771.00,218800.0,7166.50,2017-06-30,0.0431,13473.06,20639.56,ST05,West
1,S02,2017-10-05,5174670,B074,WAT,CU00335495,1,198600.0,198600.0,39600.0,...,34781.25,109391.05,159000.0,14827.70,2017-08-21,0.0431,8559.66,23387.36,ST05,West
2,S02,2017-11-14,5159070,B083,WAT,CU00338567,1,108400.0,108400.0,19500.0,...,19446.87,72033.90,88900.0,-2580.77,2014-02-28,0.0631,6802.18,4221.41,ST05,West
3,S02,2017-11-27,5161618,B100,WAT,CU00342623,1,29800.0,29800.0,4020.0,...,3932.54,17584.95,25780.0,4262.51,2014-05-31,0.0695,1952.95,6215.46,ST05,West
4,S02,2017-12-15,5180287,B150,WAT,CU00345861,1,32200.0,32200.0,6440.0,...,3929.49,11065.90,25760.0,10764.61,2017-05-06,NaN,NaN,10764.61,ST05,West


In [38]:
all_sales.dtypes

Store No                          object
Date                      datetime64[ns]
Item No                            int64
Brand                             object
Department                        object
Customer No                       object
Quantity                           int64
Price                            float64
Total Price                      float64
Line Discount Amount             float64
CRM Line Disc_ Amount            float64
Discount Amount                    int64
Tax Amount                       float64
Cost Amount                      float64
Billing                          float64
Contribution                     float64
Receipt Date              datetime64[ns]
Trade Incentive                  float64
Trade Incentives Value           float64
Total Contribution               float64
State                             object
Region                            object
dtype: object

In [39]:
import time
from datetime import datetime, date, time, timedelta

#file_week = os.path.basename(data_dir + 'Encoded/Closing Stock as on_01.12.19.xlsx' )
#date_1 = datetime.datetime.strptime(file_week, format='%d.%m.%Y')

file_week = datetime.strptime(file_week[20:26]+'20'+file_week[26:28], '%d.%m.%Y') #'%m/%d/%y %H:%M:%S')
type(file_week)
end_date = file_week + timedelta(days=7)
print(file_week)
print(end_date)


2019-12-01 00:00:00
2019-12-08 00:00:00


In [40]:
pd.to_datetime(final_summ_file['Week_Start'], format='%d.%m.%Y')
temp_df_date_range = all_sales[(all_sales['Date'] > file_week) & (all_sales['Date'] <= end_date)]



In [41]:
print(temp_df_date_range.shape)
pd.to_datetime(temp_df_date_range['Receipt Date'], format='%d.%m.%Y')
pd.to_datetime(temp_df_date_range['Date'], format='%d.%m.%Y')
print(temp_df_date_range.dtypes)
temp_df_date_range[temp_df_date_range['Item No'] == 5136095]


(2375, 22)
Store No                          object
Date                      datetime64[ns]
Item No                            int64
Brand                             object
Department                        object
Customer No                       object
Quantity                           int64
Price                            float64
Total Price                      float64
Line Discount Amount             float64
CRM Line Disc_ Amount            float64
Discount Amount                    int64
Tax Amount                       float64
Cost Amount                      float64
Billing                          float64
Contribution                     float64
Receipt Date              datetime64[ns]
Trade Incentive                  float64
Trade Incentives Value           float64
Total Contribution               float64
State                             object
Region                            object
dtype: object


,Store No,Date,Item No,Brand,Department,Customer No,Quantity,Price,Total Price,Line Discount Amount,...,Tax Amount,Cost Amount,Billing,Contribution,Receipt Date,Trade Incentive,Trade Incentives Value,Total Contribution,State,Region
73,S02,2019-12-07,5136095,B100,WAT,CU00441189,1,26150.0,26150.0,0.0,...,3988.98,15922.65,26150.0,6238.37,2018-12-07,0.0169,441.935,6680.305,ST05,West
22497,S10,2019-12-05,5136095,B100,WAT,CU00440855,1,26150.0,26150.0,0.0,...,3988.98,15922.66,26150.0,6238.36,2019-01-30,0.0169,441.935,6680.295,ST12,West
35578,S127,2019-12-02,5136095,B100,WAT,CU00440400,1,28500.0,28500.0,0.0,...,4347.46,16756.73,28500.0,7395.81,2019-07-03,0.0169,481.650,7877.460,ST21,East


In [42]:
#COl - 20 Sales QUantity
sales_quantitydf = temp_df_date_range.groupby(["Item No","Store No"])['Quantity'].agg(['sum']) #count()
#sales_quantitydf = temp_df_date_range.groupby(["Item No"])['Quantity'].agg(['sum']) #count()
print(sales_quantitydf.shape)
print(type(sales_quantitydf))
#print(sales_quantitydf.loc[:,:])


(1460, 1)
<class 'pandas.core.frame.DataFrame'>


In [43]:
sales_quantitydf.head()

sum
Item No Store No     
5100392 S109        1
5107207 S99         1
5110883 S110        7
        S118        1
5110884 S109        1

In [44]:
#merged_sales_quantity_df = final_summ_file.merge(sales_quantitydf,how = 'inner',
#                                                 left_on=["Item No_"], right_on=["Item No"],
#                                        suffixes= ('_m', '_n')) #, indicator= True)

merged_sales_quantity_df = final_summ_file.merge(sales_quantitydf, right_index=True, how = 'left',
                                                 left_on=["Item No_","Location Code"], right_on=["Item No","Store No"],
                                        suffixes= ('_m', '_n')) #, indicator= True)

merged_sales_quantity_df.rename(columns={'sum': 'sales_quantity'}, inplace=True)
merged_sales_quantity_df.head()



,Col_PK,Item No_,Location Code,Purchase MRP,Prevailing MRP as on Stock Date,State_m,Region_m,stock_quantity,days_in_stock_agg,Week_Start,...,Region_n,No_,Brand,Movement,Case Material,Watchband _ Strap Type,Dial Color,Case Shape,Case Size,sales_quantity
0,5106817_S02,5106817,S02,63140.0,66300.0,ST05,West,1,2166 days,2019-12-01,...,NaN,5106817,B063,QUARTZ,STEEL,STEEL,WHITE,RECTANGULAR,32 mm,NaN
1,5106817_S23,5106817,S23,62500.0,66300.0,ST12,West,1,2358 days,2019-12-01,...,NaN,5106817,B063,QUARTZ,STEEL,STEEL,WHITE,RECTANGULAR,32 mm,NaN
2,5106817_S24,5106817,S24,48100.0,66300.0,ST16,South,1,2128 days,2019-12-01,...,NaN,5106817,B063,QUARTZ,STEEL,STEEL,WHITE,RECTANGULAR,32 mm,NaN
3,5106817_S62,5106817,S62,56500.0,66300.0,ST02,North,1,2305 days,2019-12-01,...,NaN,5106817,B063,QUARTZ,STEEL,STEEL,WHITE,RECTANGULAR,32 mm,NaN
4,5107206_S02,5107206,S02,142500.0,154100.0,ST05,West,1,261 days,2019-12-01,...,NaN,5107206,B063,AUTOMATIC,STEEL,LEATHER,SILVER,ROUND,40 mm,NaN


In [45]:
final_summ_file  = merged_sales_quantity_df
final_summ_file[final_summ_file['Item No_'] == 5136095]

,Col_PK,Item No_,Location Code,Purchase MRP,Prevailing MRP as on Stock Date,State_m,Region_m,stock_quantity,days_in_stock_agg,Week_Start,...,Region_n,No_,Brand,Movement,Case Material,Watchband _ Strap Type,Dial Color,Case Shape,Case Size,sales_quantity
328,5136095_S02,5136095,S02,26150.0,30800.0,ST05,West,1,359 days,2019-12-01,...,NaN,5136095,B100,QUARTZ,GOLD PLATED,LEATHER,WHITE,ROUND,42 mm,1.0
329,5136095_S06,5136095,S06,28500.0,30800.0,ST09,South,1,215 days,2019-12-01,...,NaN,5136095,B100,QUARTZ,GOLD PLATED,LEATHER,WHITE,ROUND,42 mm,NaN
330,5136095_S10,5136095,S10,26150.0,30800.0,ST12,West,1,305 days,2019-12-01,...,NaN,5136095,B100,QUARTZ,GOLD PLATED,LEATHER,WHITE,ROUND,42 mm,1.0
331,5136095_S102,5136095,S102,28500.0,30800.0,ST24,North,1,95 days,2019-12-01,...,NaN,5136095,B100,QUARTZ,GOLD PLATED,LEATHER,WHITE,ROUND,42 mm,NaN
332,5136095_S109,5136095,S109,26150.0,30800.0,ST12,West,1,353 days,2019-12-01,...,NaN,5136095,B100,QUARTZ,GOLD PLATED,LEATHER,WHITE,ROUND,42 mm,NaN
333,5136095_S11,5136095,S11,28500.0,30800.0,ST13,North,2,69 days,2019-12-01,...,NaN,5136095,B100,QUARTZ,GOLD PLATED,LEATHER,WHITE,ROUND,42 mm,NaN
334,5136095_S110,5136095,S110,26150.0,30800.0,ST03,North,1,356 days,2019-12-01,...,NaN,5136095,B100,QUARTZ,GOLD PLATED,LEATHER,WHITE,ROUND,42 mm,NaN
335,5136095_S114,5136095,S114,26150.0,30800.0,ST17,South,1,409 days,2019-12-01,...,NaN,5136095,B100,QUARTZ,GOLD PLATED,LEATHER,WHITE,ROUND,42 mm,NaN
336,5136095_S120,5136095,S120,28500.0,30800.0,ST16,South,2,273 days,2019-12-01,...,NaN,5136095,B100,QUARTZ,GOLD PLATED,LEATHER,WHITE,ROUND,42 mm,NaN
337,5136095_S121,5136095,S121,28500.0,30800.0,ST28,East,1,123 days,2019-12-01,...,NaN,5136095,B100,QUARTZ,GOLD PLATED,LEATHER,WHITE,ROUND,42 mm,NaN


In [46]:
#COl - Days_to_Sell
#find the different between receipt_date which is purchase date and the Data field which is sale date
temp_df_date_range['sale_diff'] = temp_df_date_range.loc[:,'Date'] -  temp_df_date_range.loc[:,'Receipt Date'] 
#instead of agg, have to to use this logic to avoid error of aggreagte timedelta type
Days_to_Sell_Aggdf = temp_df_date_range.groupby(["Item No","Store No"])['sale_diff'].agg(['sum', 'count']) 
Days_to_Sell_Aggdf['days_to_sell'] = Days_to_Sell_Aggdf['sum']/Days_to_Sell_Aggdf['count']
Days_to_Sell_Aggdf.head()

/Users/nirajkulkarni/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


sum  count days_to_sell
Item No Store No                              
5100392 S109     1170 days      1    1170 days
5107207 S99       753 days      1     753 days
5110883 S110      749 days      7     107 days
        S118       77 days      1      77 days
5110884 S109       87 days      1      87 days

In [47]:
try:
    merged_days_sell_df = final_summ_file.merge(Days_to_Sell_Aggdf['days_to_sell'], right_index=True, how = 'left',
                                                     left_on=["Item No_","Location Code"], right_on=["Item No","Store No"],
                                            suffixes= ('_x', '_y')) #, indicator= True)
    final_summ_file  = merged_days_sell_df
except:
    print("No Data for week : " + str(file_week))
    final_summ_file['days_to_sell'] = None


In [48]:

final_summ_file[final_summ_file['Item No_'] == 5136095]

,Col_PK,Item No_,Location Code,Purchase MRP,Prevailing MRP as on Stock Date,State_m,Region_m,stock_quantity,days_in_stock_agg,Week_Start,...,No_,Brand,Movement,Case Material,Watchband _ Strap Type,Dial Color,Case Shape,Case Size,sales_quantity,days_to_sell
328,5136095_S02,5136095,S02,26150.0,30800.0,ST05,West,1,359 days,2019-12-01,...,5136095,B100,QUARTZ,GOLD PLATED,LEATHER,WHITE,ROUND,42 mm,1.0,365 days
329,5136095_S06,5136095,S06,28500.0,30800.0,ST09,South,1,215 days,2019-12-01,...,5136095,B100,QUARTZ,GOLD PLATED,LEATHER,WHITE,ROUND,42 mm,NaN,NaT
330,5136095_S10,5136095,S10,26150.0,30800.0,ST12,West,1,305 days,2019-12-01,...,5136095,B100,QUARTZ,GOLD PLATED,LEATHER,WHITE,ROUND,42 mm,1.0,309 days
331,5136095_S102,5136095,S102,28500.0,30800.0,ST24,North,1,95 days,2019-12-01,...,5136095,B100,QUARTZ,GOLD PLATED,LEATHER,WHITE,ROUND,42 mm,NaN,NaT
332,5136095_S109,5136095,S109,26150.0,30800.0,ST12,West,1,353 days,2019-12-01,...,5136095,B100,QUARTZ,GOLD PLATED,LEATHER,WHITE,ROUND,42 mm,NaN,NaT
333,5136095_S11,5136095,S11,28500.0,30800.0,ST13,North,2,69 days,2019-12-01,...,5136095,B100,QUARTZ,GOLD PLATED,LEATHER,WHITE,ROUND,42 mm,NaN,NaT
334,5136095_S110,5136095,S110,26150.0,30800.0,ST03,North,1,356 days,2019-12-01,...,5136095,B100,QUARTZ,GOLD PLATED,LEATHER,WHITE,ROUND,42 mm,NaN,NaT
335,5136095_S114,5136095,S114,26150.0,30800.0,ST17,South,1,409 days,2019-12-01,...,5136095,B100,QUARTZ,GOLD PLATED,LEATHER,WHITE,ROUND,42 mm,NaN,NaT
336,5136095_S120,5136095,S120,28500.0,30800.0,ST16,South,2,273 days,2019-12-01,...,5136095,B100,QUARTZ,GOLD PLATED,LEATHER,WHITE,ROUND,42 mm,NaN,NaT
337,5136095_S121,5136095,S121,28500.0,30800.0,ST28,East,1,123 days,2019-12-01,...,5136095,B100,QUARTZ,GOLD PLATED,LEATHER,WHITE,ROUND,42 mm,NaN,NaT


In [49]:
#Col - 21 AVerage BIlling Value
avg_sales_billing = temp_df_date_range.groupby(["Item No","Store No"])['Billing'].agg(['mean']) #count()
print(avg_sales_billing.shape)
print(type(avg_sales_billing))
#print(sales_quantitydf.loc[:,:])


(1460, 1)
<class 'pandas.core.frame.DataFrame'>


In [50]:
avg_sales_billing.head()

mean
Item No Store No          
5100392 S109      122000.0
5107207 S99       115600.0
5110883 S110       19716.0
        S118       25500.0
5110884 S109       23500.0

In [51]:
merged_avg_billing_df = final_summ_file.merge(avg_sales_billing, right_index=True, how = 'left',
                                                 left_on=["Item No_","Location Code"], right_on=["Item No","Store No"],
                                        suffixes= ('_m', '_n')) #, indicator= True)

merged_avg_billing_df.rename(columns={'mean': 'avg_billing'}, inplace=True)
merged_avg_billing_df.head()

,Col_PK,Item No_,Location Code,Purchase MRP,Prevailing MRP as on Stock Date,State_m,Region_m,stock_quantity,days_in_stock_agg,Week_Start,...,Brand,Movement,Case Material,Watchband _ Strap Type,Dial Color,Case Shape,Case Size,sales_quantity,days_to_sell,avg_billing
0,5106817_S02,5106817,S02,63140.0,66300.0,ST05,West,1,2166 days,2019-12-01,...,B063,QUARTZ,STEEL,STEEL,WHITE,RECTANGULAR,32 mm,NaN,NaT,NaN
1,5106817_S23,5106817,S23,62500.0,66300.0,ST12,West,1,2358 days,2019-12-01,...,B063,QUARTZ,STEEL,STEEL,WHITE,RECTANGULAR,32 mm,NaN,NaT,NaN
2,5106817_S24,5106817,S24,48100.0,66300.0,ST16,South,1,2128 days,2019-12-01,...,B063,QUARTZ,STEEL,STEEL,WHITE,RECTANGULAR,32 mm,NaN,NaT,NaN
3,5106817_S62,5106817,S62,56500.0,66300.0,ST02,North,1,2305 days,2019-12-01,...,B063,QUARTZ,STEEL,STEEL,WHITE,RECTANGULAR,32 mm,NaN,NaT,NaN
4,5107206_S02,5107206,S02,142500.0,154100.0,ST05,West,1,261 days,2019-12-01,...,B063,AUTOMATIC,STEEL,LEATHER,SILVER,ROUND,40 mm,NaN,NaT,NaN


In [52]:
final_summ_file  = merged_avg_billing_df
#Replace NaN with 0
final_summ_file[final_summ_file['Item No_'] == 5136095]

,Col_PK,Item No_,Location Code,Purchase MRP,Prevailing MRP as on Stock Date,State_m,Region_m,stock_quantity,days_in_stock_agg,Week_Start,...,Brand,Movement,Case Material,Watchband _ Strap Type,Dial Color,Case Shape,Case Size,sales_quantity,days_to_sell,avg_billing
328,5136095_S02,5136095,S02,26150.0,30800.0,ST05,West,1,359 days,2019-12-01,...,B100,QUARTZ,GOLD PLATED,LEATHER,WHITE,ROUND,42 mm,1.0,365 days,26150.0
329,5136095_S06,5136095,S06,28500.0,30800.0,ST09,South,1,215 days,2019-12-01,...,B100,QUARTZ,GOLD PLATED,LEATHER,WHITE,ROUND,42 mm,NaN,NaT,NaN
330,5136095_S10,5136095,S10,26150.0,30800.0,ST12,West,1,305 days,2019-12-01,...,B100,QUARTZ,GOLD PLATED,LEATHER,WHITE,ROUND,42 mm,1.0,309 days,26150.0
331,5136095_S102,5136095,S102,28500.0,30800.0,ST24,North,1,95 days,2019-12-01,...,B100,QUARTZ,GOLD PLATED,LEATHER,WHITE,ROUND,42 mm,NaN,NaT,NaN
332,5136095_S109,5136095,S109,26150.0,30800.0,ST12,West,1,353 days,2019-12-01,...,B100,QUARTZ,GOLD PLATED,LEATHER,WHITE,ROUND,42 mm,NaN,NaT,NaN
333,5136095_S11,5136095,S11,28500.0,30800.0,ST13,North,2,69 days,2019-12-01,...,B100,QUARTZ,GOLD PLATED,LEATHER,WHITE,ROUND,42 mm,NaN,NaT,NaN
334,5136095_S110,5136095,S110,26150.0,30800.0,ST03,North,1,356 days,2019-12-01,...,B100,QUARTZ,GOLD PLATED,LEATHER,WHITE,ROUND,42 mm,NaN,NaT,NaN
335,5136095_S114,5136095,S114,26150.0,30800.0,ST17,South,1,409 days,2019-12-01,...,B100,QUARTZ,GOLD PLATED,LEATHER,WHITE,ROUND,42 mm,NaN,NaT,NaN
336,5136095_S120,5136095,S120,28500.0,30800.0,ST16,South,2,273 days,2019-12-01,...,B100,QUARTZ,GOLD PLATED,LEATHER,WHITE,ROUND,42 mm,NaN,NaT,NaN
337,5136095_S121,5136095,S121,28500.0,30800.0,ST28,East,1,123 days,2019-12-01,...,B100,QUARTZ,GOLD PLATED,LEATHER,WHITE,ROUND,42 mm,NaN,NaT,NaN


In [53]:
final_summ_file.columns

Index(['Col_PK', 'Item No_', 'Location Code', 'Purchase MRP',
       'Prevailing MRP as on Stock Date', 'State_m', 'Region_m',
       'stock_quantity', 'days_in_stock_agg', 'Week_Start', 'Month', 'Year',
       'Store Code', 'Store type', 'Store location', 'City type', 'State_n',
       'Region_n', 'No_', 'Brand', 'Movement', 'Case Material',
       'Watchband _ Strap Type', 'Dial Color', 'Case Shape', 'Case Size',
       'sales_quantity', 'days_to_sell', 'avg_billing'],
      dtype='object')

In [54]:
#read all purchase files
purchase_17_18 = pd.read_excel(data_dir + 'item-data/2017-18/Purchase Data 01.04.17 to 31.03.18.xlsx')

In [55]:
purchase_18_19 = pd.read_excel(data_dir + 'item-data/2018-19/Purchase Data 01.04.18 to 31.03.19.xlsx')

In [56]:
purchase_19_20 = pd.read_excel(data_dir + 'item-data/2019-20/Purchase Data 01.04.19 to 06.02.2020.xlsx')

In [57]:
print(purchase_17_18.shape)
print(purchase_18_19.shape)
print(purchase_19_20.shape)

(92573, 10)
(92739, 10)
(51784, 10)


In [58]:
#concat all sales file
all_purchase_list = [purchase_17_18,purchase_18_19,purchase_19_20]
all_purchase = pd.concat(all_purchase_list)
#all_purchase['Item No_'] = all_purchase["Item No_"].astype(int)
all_purchase.dtypes

Location Code            object
Posting Date     datetime64[ns]
Item No_                float64
Brand                    object
Department               object
Quantity                float64
Purchase MRP            float64
Cost Amount             float64
State Code               object
Region                   object
dtype: object

In [59]:
all_purchase[(all_purchase['Item No_'] == 5136095) & (all_purchase['Location Code'] == 'S06')]


,Location Code,Posting Date,Item No_,Brand,Department,Quantity,Purchase MRP,Cost Amount,State Code,Region
2989,S06,2017-04-21,5136095.0,B100,WAT,1.0,27100.0,17721.42,ST09,South
28360,S06,2017-08-19,5136095.0,B100,WAT,1.0,27100.0,14926.18,ST09,South
35482,S06,2017-09-18,5136095.0,B100,WAT,1.0,27100.0,14897.56,ST09,South
35483,S06,2017-09-18,5136095.0,B100,WAT,1.0,27100.0,14897.56,ST09,South
50829,S06,2017-11-07,5136095.0,B100,WAT,1.0,27100.0,14926.17,ST09,South
50830,S06,2017-11-07,5136095.0,B100,WAT,1.0,27100.0,14926.17,ST09,South
27656,S06,2018-06-30,5136095.0,B100,WAT,1.0,26150.0,15904.58,ST09,South
34843,S06,2018-07-31,5136095.0,B100,WAT,1.0,26150.0,15623.38,ST09,South
42081,S06,2018-08-31,5136095.0,B100,WAT,1.0,26150.0,15908.88,ST09,South
42082,S06,2018-08-31,5136095.0,B100,WAT,1.0,26150.0,15908.88,ST09,South


In [60]:
purchase_temp_df_date_range = all_purchase[(all_purchase['Posting Date'] > file_week) & (all_purchase['Posting Date'] <= end_date)]
purchase_temp_df_date_range.head(3)

,Location Code,Posting Date,Item No_,Brand,Department,Quantity,Purchase MRP,Cost Amount,State Code,Region
40811,S133,2019-12-04,5193497.0,B017,WAT,1.0,4995.0,2635.0,ST07,North
40812,S133,2019-12-04,5193497.0,B017,WAT,1.0,4995.0,2635.0,ST07,North
40813,S133,2019-12-04,5193497.0,B017,WAT,1.0,4995.0,2635.0,ST07,North


In [61]:
purchase_quantitydf = purchase_temp_df_date_range.groupby(["Item No_","Location Code"])['Quantity'].agg(['sum'])
print(type(purchase_quantitydf))
purchase_quantitydf.head()

<class 'pandas.core.frame.DataFrame'>


sum
Item No_  Location Code     
5107292.0 S102           1.0
5110883.0 S120           1.0
5110884.0 S07            1.0
          S120           1.0
          S140           1.0

In [62]:
merged_purchase_quantity_df = final_summ_file.merge(purchase_quantitydf, right_index=True, how = 'left',
                                                 left_on=["Item No_","Location Code"], right_on=["Item No_","Location Code"],
                                        suffixes= ('_i', '_j')) #, indicator= True)

merged_purchase_quantity_df.rename(columns={'sum': 'purchase_quantity'}, inplace=True)
merged_purchase_quantity_df.head()


,Col_PK,Item No_,Location Code,Purchase MRP,Prevailing MRP as on Stock Date,State_m,Region_m,stock_quantity,days_in_stock_agg,Week_Start,...,Movement,Case Material,Watchband _ Strap Type,Dial Color,Case Shape,Case Size,sales_quantity,days_to_sell,avg_billing,purchase_quantity
0,5106817_S02,5106817,S02,63140.0,66300.0,ST05,West,1,2166 days,2019-12-01,...,QUARTZ,STEEL,STEEL,WHITE,RECTANGULAR,32 mm,NaN,NaT,NaN,NaN
1,5106817_S23,5106817,S23,62500.0,66300.0,ST12,West,1,2358 days,2019-12-01,...,QUARTZ,STEEL,STEEL,WHITE,RECTANGULAR,32 mm,NaN,NaT,NaN,NaN
2,5106817_S24,5106817,S24,48100.0,66300.0,ST16,South,1,2128 days,2019-12-01,...,QUARTZ,STEEL,STEEL,WHITE,RECTANGULAR,32 mm,NaN,NaT,NaN,NaN
3,5106817_S62,5106817,S62,56500.0,66300.0,ST02,North,1,2305 days,2019-12-01,...,QUARTZ,STEEL,STEEL,WHITE,RECTANGULAR,32 mm,NaN,NaT,NaN,NaN
4,5107206_S02,5107206,S02,142500.0,154100.0,ST05,West,1,261 days,2019-12-01,...,AUTOMATIC,STEEL,LEATHER,SILVER,ROUND,40 mm,NaN,NaT,NaN,NaN


In [63]:
final_summ_file = merged_purchase_quantity_df

final_summ_file[final_summ_file['Item No_'] == 5136095]

,Col_PK,Item No_,Location Code,Purchase MRP,Prevailing MRP as on Stock Date,State_m,Region_m,stock_quantity,days_in_stock_agg,Week_Start,...,Movement,Case Material,Watchband _ Strap Type,Dial Color,Case Shape,Case Size,sales_quantity,days_to_sell,avg_billing,purchase_quantity
328,5136095_S02,5136095,S02,26150.0,30800.0,ST05,West,1,359 days,2019-12-01,...,QUARTZ,GOLD PLATED,LEATHER,WHITE,ROUND,42 mm,1.0,365 days,26150.0,NaN
329,5136095_S06,5136095,S06,28500.0,30800.0,ST09,South,1,215 days,2019-12-01,...,QUARTZ,GOLD PLATED,LEATHER,WHITE,ROUND,42 mm,NaN,NaT,NaN,NaN
330,5136095_S10,5136095,S10,26150.0,30800.0,ST12,West,1,305 days,2019-12-01,...,QUARTZ,GOLD PLATED,LEATHER,WHITE,ROUND,42 mm,1.0,309 days,26150.0,NaN
331,5136095_S102,5136095,S102,28500.0,30800.0,ST24,North,1,95 days,2019-12-01,...,QUARTZ,GOLD PLATED,LEATHER,WHITE,ROUND,42 mm,NaN,NaT,NaN,NaN
332,5136095_S109,5136095,S109,26150.0,30800.0,ST12,West,1,353 days,2019-12-01,...,QUARTZ,GOLD PLATED,LEATHER,WHITE,ROUND,42 mm,NaN,NaT,NaN,NaN
333,5136095_S11,5136095,S11,28500.0,30800.0,ST13,North,2,69 days,2019-12-01,...,QUARTZ,GOLD PLATED,LEATHER,WHITE,ROUND,42 mm,NaN,NaT,NaN,NaN
334,5136095_S110,5136095,S110,26150.0,30800.0,ST03,North,1,356 days,2019-12-01,...,QUARTZ,GOLD PLATED,LEATHER,WHITE,ROUND,42 mm,NaN,NaT,NaN,NaN
335,5136095_S114,5136095,S114,26150.0,30800.0,ST17,South,1,409 days,2019-12-01,...,QUARTZ,GOLD PLATED,LEATHER,WHITE,ROUND,42 mm,NaN,NaT,NaN,NaN
336,5136095_S120,5136095,S120,28500.0,30800.0,ST16,South,2,273 days,2019-12-01,...,QUARTZ,GOLD PLATED,LEATHER,WHITE,ROUND,42 mm,NaN,NaT,NaN,NaN
337,5136095_S121,5136095,S121,28500.0,30800.0,ST28,East,1,123 days,2019-12-01,...,QUARTZ,GOLD PLATED,LEATHER,WHITE,ROUND,42 mm,NaN,NaT,NaN,NaN


In [64]:
final_summ_file[['sales_quantity','days_to_sell','avg_billing','purchase_quantity','stock_quantity']] = final_summ_file[['sales_quantity','days_to_sell','avg_billing','purchase_quantity','stock_quantity']].fillna(0)

/Users/nirajkulkarni/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Passing integers to fillna is deprecated, will raise a TypeError in a future version.  To retain the old behavior, pass pd.Timedelta(seconds=n) instead.
  """Entry point for launching an IPython kernel.


In [65]:
#Stock Qty + purchase Qty
final_summ_file['total_available_qty'] = final_summ_file['stock_quantity'] + final_summ_file['purchase_quantity']

In [66]:
final_summ_file[final_summ_file['Item No_'] == 5136095]

,Col_PK,Item No_,Location Code,Purchase MRP,Prevailing MRP as on Stock Date,State_m,Region_m,stock_quantity,days_in_stock_agg,Week_Start,...,Case Material,Watchband _ Strap Type,Dial Color,Case Shape,Case Size,sales_quantity,days_to_sell,avg_billing,purchase_quantity,total_available_qty
328,5136095_S02,5136095,S02,26150.0,30800.0,ST05,West,1,359 days,2019-12-01,...,GOLD PLATED,LEATHER,WHITE,ROUND,42 mm,1.0,365 days,26150.0,0.0,1.0
329,5136095_S06,5136095,S06,28500.0,30800.0,ST09,South,1,215 days,2019-12-01,...,GOLD PLATED,LEATHER,WHITE,ROUND,42 mm,0.0,0 days,0.0,0.0,1.0
330,5136095_S10,5136095,S10,26150.0,30800.0,ST12,West,1,305 days,2019-12-01,...,GOLD PLATED,LEATHER,WHITE,ROUND,42 mm,1.0,309 days,26150.0,0.0,1.0
331,5136095_S102,5136095,S102,28500.0,30800.0,ST24,North,1,95 days,2019-12-01,...,GOLD PLATED,LEATHER,WHITE,ROUND,42 mm,0.0,0 days,0.0,0.0,1.0
332,5136095_S109,5136095,S109,26150.0,30800.0,ST12,West,1,353 days,2019-12-01,...,GOLD PLATED,LEATHER,WHITE,ROUND,42 mm,0.0,0 days,0.0,0.0,1.0
333,5136095_S11,5136095,S11,28500.0,30800.0,ST13,North,2,69 days,2019-12-01,...,GOLD PLATED,LEATHER,WHITE,ROUND,42 mm,0.0,0 days,0.0,0.0,2.0
334,5136095_S110,5136095,S110,26150.0,30800.0,ST03,North,1,356 days,2019-12-01,...,GOLD PLATED,LEATHER,WHITE,ROUND,42 mm,0.0,0 days,0.0,0.0,1.0
335,5136095_S114,5136095,S114,26150.0,30800.0,ST17,South,1,409 days,2019-12-01,...,GOLD PLATED,LEATHER,WHITE,ROUND,42 mm,0.0,0 days,0.0,0.0,1.0
336,5136095_S120,5136095,S120,28500.0,30800.0,ST16,South,2,273 days,2019-12-01,...,GOLD PLATED,LEATHER,WHITE,ROUND,42 mm,0.0,0 days,0.0,0.0,2.0
337,5136095_S121,5136095,S121,28500.0,30800.0,ST28,East,1,123 days,2019-12-01,...,GOLD PLATED,LEATHER,WHITE,ROUND,42 mm,0.0,0 days,0.0,0.0,1.0
